In [2]:
# Cell 1: Import necessary libraries
import numpy as np
import music21
from tensorflow.keras.models import load_model

In [3]:
# Cell 2: Helper functions
def parse_midi_file(file_path):
    """Extract notes and chords from a MIDI file."""
    try:
        midi_stream = music21.converter.parse(file_path)
        notes = []
        for element in midi_stream.flat.notes:
            if isinstance(element, music21.note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, music21.chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
        return notes
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return []

def create_midi_file(predicted_notes, output_path):
    """Create a MIDI file from predicted notes."""
    offset = 0
    output_notes = []

    for note in predicted_notes:
        if '.' in note or note.isdigit():
            # Chord
            chord_notes = [music21.note.Note(int(n)) for n in note.split('.')]
            new_chord = music21.chord.Chord(chord_notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            # Note
            new_note = music21.note.Note(note)
            new_note.offset = offset
            output_notes.append(new_note)

        offset += 0.5  # Add a fixed step for note spacing

    midi_stream = music21.stream.Stream(output_notes)
    midi_stream.write('midi', fp=output_path)

def prepare_input_sequence(notes, note_to_int, sequence_length):
    """Prepare the input sequence for the model."""
    int_sequence = [note_to_int[note] for note in notes if note in note_to_int]
    pattern = int_sequence[-sequence_length:]
    input_sequence = np.reshape(pattern, (1, sequence_length, 1))
    input_sequence = input_sequence / float(len(note_to_int))
    return input_sequence

In [4]:
# Cell 3: Load the trained model and prepare data
model_path = 'final_model.keras'  # Path to the saved model
model = load_model(model_path)
print(f"Model loaded from {model_path}.")

input_midi_path = 'seed5.mid'  # Path to the input MIDI file
output_midi_path = 'generated_variation.mid'  # Path to save the generated MIDI variation

# Load and process the input MIDI file
sequence_length = 50  # Same sequence length as used in training
input_notes = parse_midi_file(input_midi_path)

if not input_notes:
    raise ValueError("No valid notes extracted from the input MIDI file.")

# Generate mapping from notes to integers and vice versa
unique_notes = sorted(set(input_notes))
note_to_int = {note: num for num, note in enumerate(unique_notes)}
int_to_note = {num: note for note, num in note_to_int.items()}

print(f"Unique notes in input: {len(unique_notes)}")

I0000 00:00:1733244066.030568 1194009 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46872 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:09:00.0, compute capability: 8.6


Model loaded from final_model.keras.
Unique notes in input: 33


/usr/local/lib/python3.10/dist-packages/music21/stream/base.py:3689: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


In [5]:
# Cell 4: Generate new music
def generate_variation(model, input_notes, note_to_int, int_to_note, sequence_length, num_notes):
    """Generate a variation of music based on the trained model."""
    generated_notes = []
    
    # Ensure the input_notes size matches the sequence length
    input_notes = input_notes[-sequence_length:]  # Truncate to the last 'sequence_length' notes
    
    # Convert input_notes to integer representations
    input_notes_int = [note_to_int[note] for note in input_notes]
    
    # Reshape the input sequence to be compatible with the model
    input_sequence = np.array(input_notes_int)
    input_sequence = input_sequence.reshape((1, sequence_length, 1))  # Reshaping for LSTM model
    
    # Generate notes
    for _ in range(num_notes):
        # Get model prediction
        prediction = model.predict(input_sequence, verbose=0)
        
        # Get the index with the highest probability (as it is a classification problem)
        predicted_index = np.argmax(prediction)
        
        # Wrap around the predicted index to stay within bounds of available notes
        predicted_index = predicted_index % len(int_to_note)
        
        # Get the predicted note from int_to_note using the wrapped index
        predicted_note = int_to_note[predicted_index]
        generated_notes.append(predicted_note)
        
        # Update the input sequence for the next prediction (append the predicted index)
        # Reshape predicted_index to match the 3D shape of the input sequence
        predicted_index_reshaped = np.array([[predicted_index]]).reshape((1, 1, 1))
        input_sequence = np.append(input_sequence[:, 1:, :], predicted_index_reshaped, axis=1)
    
    return generated_notes

num_notes_to_generate = 200  # Number of notes to generate
generated_notes = generate_variation(model, input_notes, note_to_int, int_to_note, sequence_length, num_notes_to_generate)
print(f"Generated {len(generated_notes)} new notes.")


I0000 00:00:1733244074.082407 1194341 cuda_dnn.cc:529] Loaded cuDNN version 90300


Generated 200 new notes.


In [6]:
# Cell 5: Save the generated variation to a MIDI file
create_midi_file(generated_notes, output_midi_path)
print(f"Generated MIDI variation saved to {output_midi_path}.")

Generated MIDI variation saved to generated_variation.mid.
